# Test Atlas v2

See atlas v1 issues:
https://github.com/roocs/clisops/issues/317

In [1]:
import xarray as xr
from clisops.core import subset
from clisops.ops import subset as subset_op

import time
import os

In [2]:
basedir_atlas_v2 = "/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset"

cmip6_nc = f"{basedir_atlas_v2}/CMIP6/historical/t_CMIP6_historical_mon_185001-201412_v02.nc"
cmip6_nc

'/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset/CMIP6/historical/t_CMIP6_historical_mon_185001-201412_v02.nc'

## ncdump - CMIP6

In [3]:
! du -sh {cmip6_nc}

5.6G	/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset/CMIP6/historical/t_CMIP6_historical_mon_185001-201412_v02.nc


In [4]:
! ncdump -sh {cmip6_nc}

netcdf t_CMIP6_historical_mon_185001-201412_v02 {
dimensions:
	bnds = 2 ;
	lon = 360 ;
	lat = 180 ;
	time = 1980 ;
	member = 30 ;
variables:
	double lat(lat) ;
		lat:standard_name = "latitude" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
		lat:long_name = "latitude" ;
		lat:bounds = "lat_bnds" ;
		lat:_Storage = "chunked" ;
		lat:_ChunkSizes = 180 ;
		lat:_Fletcher32 = "true" ;
		lat:_Shuffle = "true" ;
		lat:_DeflateLevel = 1 ;
		lat:_Endianness = "little" ;
	double lat_bnds(lat, bnds) ;
		lat_bnds:_Storage = "chunked" ;
		lat_bnds:_ChunkSizes = 180, 2 ;
		lat_bnds:_Fletcher32 = "true" ;
		lat_bnds:_Shuffle = "true" ;
		lat_bnds:_DeflateLevel = 1 ;
		lat_bnds:_Endianness = "little" ;
	double lon(lon) ;
		lon:standard_name = "longitude" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
		lon:long_name = "longitude" ;
		lon:bounds = "lon_bnds" ;
		lon:_Storage = "chunked" ;
		lon:_ChunkSizes = 360 ;
		lon:_Fletcher32 = "true" ;
		lon:_Shuffle = "true" ;
		lon:_DeflateLevel = 1

## helper functions

In [5]:
def ds_info(ds):
    # compression levels
    print("data vars with compression:")
    for var in ds.data_vars:
        # print(var, ds[var].encoding, "\n")
        complevel = ds[var].encoding.get("complevel", 0)
        print(var, "compression level =", complevel)
        
    # fill values
    print("\nfill values:")
    var_list = list(ds.coords) + list(ds.data_vars)
    for var in var_list:
        fill_value = ds[var].encoding.get("_FillValue")
        print(var, "fill value =", fill_value)
        
    # string attributes with compression
    print("\nstring attributes with compression:")
    for cvar in [
            "member_id",
            "gcm_variant",
            "gcm_model",
            "gcm_institution",
            "rcm_variant",
            "rcm_model",
            "rcm_institution",
        ]:
            for en in ["zlib", "shuffle", "complevel"]:
                try:
                    print(cvar, en, ds[cvar].encoding[en])
                except KeyError:
                    pass

## xarray - CMIP6

TODO: still using two fill values for variable t. Without a fix it is not possible to write as netcdf file.

Error Message: 
Variable 't' has conflicting _FillValue (-1.7014118346046923e+38) and missing_value (1.0384593717069655e+34). Cannot encode data.

In [6]:
ds_cmip6 = xr.open_dataset(cmip6_nc)
ds_cmip6

/home/k/k204199/.envs/rook/lib/python3.11/site-packages/xarray/conventions.py:543: SerializationWarning: variable 't' has multiple fill values {1.0384594e+34, -1.7014118e+38}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


<xarray.Dataset>
Dimensions:          (lat: 180, bnds: 2, lon: 360, time: 1980, member: 30)
Coordinates:
  * lat              (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon              (lon) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * time             (time) datetime64[ns] 1850-01-01 1850-02-01 ... 2014-12-01
    member_id        (member) object ...
    gcm_institution  (member) object ...
    gcm_model        (member) object ...
    gcm_variant      (member) object ...
    height2m         float64 ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (time, bnds) datetime64[ns] ...
    t                (member, time, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_min:         -180.0
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-05 15:51:48.916442+01:00
    tracking_id:                4f887b4c-eaff-448c-9e50-4cacd5d16f38

In [7]:
ds_info(ds_cmip6)

data vars with compression:
lat_bnds compression level = 1
lon_bnds compression level = 1
time_bnds compression level = 1
t compression level = 1
crs compression level = 0

fill values:
lat fill value = None
lon fill value = None
time fill value = None
member_id fill value = None
gcm_institution fill value = None
gcm_model fill value = None
gcm_variant fill value = None
height2m fill value = None
lat_bnds fill value = None
lon_bnds fill value = None
time_bnds fill value = None
t fill value = -1.7014118e+38
crs fill value = None

string attributes with compression:
member_id zlib True
member_id shuffle True
member_id complevel 1
gcm_variant zlib True
gcm_variant shuffle True
gcm_variant complevel 1
gcm_model zlib True
gcm_model shuffle True
gcm_model complevel 1
gcm_institution zlib True
gcm_institution shuffle True
gcm_institution complevel 1


In [8]:
ds = ds_cmip6.isel(time=slice(0, 12), lon=slice(30, 50), lat=slice(50, 70))
ds

<xarray.Dataset>
Dimensions:          (lat: 20, bnds: 2, lon: 20, time: 12, member: 30)
Coordinates:
  * lat              (lat) float64 -39.5 -38.5 -37.5 -36.5 ... -22.5 -21.5 -20.5
  * lon              (lon) float64 -149.5 -148.5 -147.5 ... -132.5 -131.5 -130.5
  * time             (time) datetime64[ns] 1850-01-01 1850-02-01 ... 1850-12-01
    member_id        (member) object ...
    gcm_institution  (member) object ...
    gcm_model        (member) object ...
    gcm_variant      (member) object ...
    height2m         float64 ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (time, bnds) datetime64[ns] ...
    t                (member, time, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_min:         -180.0
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-05 15:51:48.916442+01:00
    tracking_id:                4f887b4c-eaff-448c-9e50-4cacd5d16f38

In [9]:
# clean up outputs

! rm /tmp/output_*

In [10]:
try:
    ds.to_netcdf("/tmp/output_atlas_v2_cmip6_xarray.nc")
except ValueError as e:
    print("Fails to write as netcdf!", e)
else:
    print("it worked!")

Fails to write as netcdf! Variable 't' has conflicting _FillValue (-1.7014118346046923e+38) and missing_value (1.0384593717069655e+34). Cannot encode data.


In [11]:
# need to apply fixes to apply operations and save to netcdf

from rook.utils import atlas_fixes

start = time.time()

ds = atlas_fixes.apply_atlas_fixes("c3s-atlas-v2.cmip6", ds)
ds.to_netcdf("/tmp/atlas_v2_cmip6_4.nc")

duration = time.time() - start
print(f"duration: {duration} secs")

debug var lat {'zlib': True, 'szip': False, 'zstd': False, 'bzip2': False, 'blosc': False, 'shuffle': True, 'complevel': 1, 'fletcher32': True, 'contiguous': False, 'chunksizes': (180,), 'source': '/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset/CMIP6/historical/t_CMIP6_historical_mon_185001-201412_v02.nc', 'original_shape': (180,), 'dtype': dtype('float64')}
debug var lon {'zlib': True, 'szip': False, 'zstd': False, 'bzip2': False, 'blosc': False, 'shuffle': True, 'complevel': 1, 'fletcher32': True, 'contiguous': False, 'chunksizes': (360,), 'source': '/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset/CMIP6/historical/t_CMIP6_historical_mon_185001-201412_v02.nc', 'original_shape': (360,), 'dtype': dtype('float64')}
debug var time {'zlib': True, 'szip': False, 'zstd': False, 'bzip2': False, 'blosc': False, 'shuffle': True, 'complevel': 1, 'fletcher32': True, 'contiguous': False, 'chunksizes': (1980,), 'source': '/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dat

## clisops-core - cmip6

INFO: clisops-core has the same issues like xarray. It does not apply any fixes. 

In [12]:
ds = subset.subset_bbox(
    ds_cmip6, lat_bnds=[45, 50], lon_bnds=[-60, -55],
    start_date='2013-01', end_date='2013-12')
ds

<xarray.Dataset>
Dimensions:          (lat: 5, bnds: 2, lon: 5, time: 12, member: 30)
Coordinates:
  * lat              (lat) float64 45.5 46.5 47.5 48.5 49.5
  * lon              (lon) float64 -59.5 -58.5 -57.5 -56.5 -55.5
  * time             (time) datetime64[ns] 2013-01-01 2013-02-01 ... 2013-12-01
    member_id        (member) object 'AS-RCEC_TaiESM1_r1i1p1f1' ... 'NUIST_NE...
    gcm_institution  (member) object 'AS-RCEC' 'AWI' ... 'NOAA-GFDL' 'NUIST'
    gcm_model        (member) object 'TaiESM1' 'AWI-CM-1-1-MR' ... 'NESM3'
    gcm_variant      (member) object 'r1i1p1f1' 'r1i1p1f1' ... 'r1i1p1f1'
    height2m         float64 2.0
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (time, bnds) datetime64[ns] ...
    t                (member, time, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/27)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-05 15:51:48.916442+01:00
    tracking_id:                4f887b4c-eaff-448c-9e50-4cacd5d16f38
    project_id:                 c3s-atlas-v2

In [13]:
try:
    ds.to_netcdf("/tmp/output_atlas_v2_cmip6_clisops_core.nc")
except ValueError as e:
    print("Fails to write as netcdf!", e)

Fails to write as netcdf! Variable 't' has conflicting _FillValue (-1.7014118346046923e+38) and missing_value (1.0384593717069655e+34). Cannot encode data.


## clisops-ops - cmip6 - subset by time

TODO: subset operation takes *very* long ... about 11 secs. It should only take much less than a second!

In [14]:
# clean up outputs

! rm /tmp/output_*

In [15]:
start = time.time()

outputs = subset_op(
    ds=ds_cmip6,
    time="2013-01-01/2013-12-30",
    # area=(0.0, 49.0, 10.0, 65.0),
    output_type="nc",
    # output_type="xarray",
    output_dir="/tmp",
    split_method="time:auto",
    file_namer="simple"
)

duration = time.time() - start
print(f"duration: {duration} secs")

print("Number of output files: ", len(outputs))
outputs[0]

/home/k/k204199/.envs/rook/lib/python3.11/site-packages/xarray/conventions.py:224: SerializationWarning: variable member_id has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  warnings.warn(
/home/k/k204199/.envs/rook/lib/python3.11/site-packages/xarray/conventions.py:224: SerializationWarning: variable gcm_institution has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  warnings.warn(
/home/k/k204199/.envs/rook/lib/python3.11/site-packages/xarray/conventions.py:224: SerializationWarning: variable gcm_model has data in the form of a dask array with dtype=object, which means it is being loaded int

duration: 11.588474035263062 secs
Number of output files:  1


'/tmp/output_001.nc'

In [16]:
file_size = os.path.getsize(outputs[0])
print("File Size is :", file_size/(1024*1024), "MB")

File Size is : 71.30526638031006 MB


In [17]:
ds = xr.open_dataset(outputs[0])
ds

<xarray.Dataset>
Dimensions:          (lat: 180, bnds: 2, lon: 360, time: 24, member: 30)
Coordinates:
  * lat              (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon              (lon) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * time             (time) datetime64[ns] 2013-01-01 2013-02-01 ... 2014-12-01
    member_id        (member) object ...
    gcm_institution  (member) object ...
    gcm_model        (member) object ...
    gcm_variant      (member) object ...
    height2m         float64 ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (time, bnds) datetime64[ns] ...
    t                (member, time, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/27)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-05 15:51:48.916442+01:00
    tracking_id:                4f887b4c-eaff-448c-9e50-4cacd5d16f38
    project_id:                 c3s-atlas-v2

In [18]:
ds_info(ds)

data vars with compression:
lat_bnds compression level = 1
lon_bnds compression level = 1
time_bnds compression level = 1
t compression level = 1
crs compression level = 0

fill values:
lat fill value = None
lon fill value = None
time fill value = None
member_id fill value = None
gcm_institution fill value = None
gcm_model fill value = None
gcm_variant fill value = None
height2m fill value = None
lat_bnds fill value = None
lon_bnds fill value = None
time_bnds fill value = None
t fill value = 1.0384594e+34
crs fill value = None

string attributes with compression:
member_id zlib False
member_id shuffle False
member_id complevel 0
gcm_variant zlib False
gcm_variant shuffle False
gcm_variant complevel 0
gcm_model zlib False
gcm_model shuffle False
gcm_model complevel 0
gcm_institution zlib False
gcm_institution shuffle False
gcm_institution complevel 0


## clisops-ops - cmip6 - subset by bbox

TODO: subset bbox is not possible! It uses a lot of memory ... even my 32GB VM was not enough for a successful run.

We need to patch `clisops.utils.dataset_utils.is_time()` ... the function `np.atleast1d` leads to this memory overflow.

In [19]:
# monkey patch for clisops

from clisops.utils import dataset_utils

def custom_is_time(coord):
    print(f"Custom behavior for is_time with input: {coord.name}")
    
    import numpy as np
    
    if "time" in coord.cf.coordinates and coord.name in coord.cf.coordinates["time"]:
        return True

    if (
        "time" in coord.cf.standard_names
        and coord.name in coord.cf.standard_names["time"]
    ):
        return True

    if np.issubdtype(coord.dtype, np.datetime64):
        return True

    # TODO: this code leads to memory overflow
    print(f"np.atleast_1d(coord.values) does not work here: {coord.name}")
    #if isinstance(np.atleast_1d(coord.values)[0], cftime.datetime):
    #    return True

    if hasattr(coord, "axis"):
        if coord.axis == "T":
            return True

    return False


# Monkey patch the function
dataset_utils.is_time = custom_is_time


In [20]:
# clean up outputs

! rm /tmp/output_*

In [21]:
from clisops.parameter import time_interval

if True:
    start = time.time()

    outputs = subset_op(
        ds=ds_cmip6,
        time=time_interval("2013-01-01", "2013-12-30"),
        area=(0.0, 50.0, 10.0, 60.0),
        output_type="nc",
        # output_type="xarray",
        output_dir="/tmp",
        split_method="time:auto",
        file_namer="simple"
    )

    duration = time.time() - start
    print(f"duration: {duration} secs")

    outputs[0]

Custom behavior for is_time with input: time
Custom behavior for is_time with input: member_id
np.atleast_1d(coord.values) does not work here: member_id
Custom behavior for is_time with input: gcm_institution
np.atleast_1d(coord.values) does not work here: gcm_institution
Custom behavior for is_time with input: gcm_model
np.atleast_1d(coord.values) does not work here: gcm_model
Custom behavior for is_time with input: gcm_variant
np.atleast_1d(coord.values) does not work here: gcm_variant
Custom behavior for is_time with input: lat_bnds
np.atleast_1d(coord.values) does not work here: lat_bnds
Custom behavior for is_time with input: lon_bnds
np.atleast_1d(coord.values) does not work here: lon_bnds
Custom behavior for is_time with input: time_bnds
Custom behavior for is_time with input: t
np.atleast_1d(coord.values) does not work here: t
Custom behavior for is_time with input: crs
np.atleast_1d(coord.values) does not work here: crs
Custom behavior for is_time with input: time
Custom behav

/home/k/k204199/.envs/rook/lib/python3.11/site-packages/xarray/conventions.py:224: SerializationWarning: variable member_id has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  warnings.warn(
/home/k/k204199/.envs/rook/lib/python3.11/site-packages/xarray/conventions.py:224: SerializationWarning: variable gcm_institution has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  warnings.warn(
/home/k/k204199/.envs/rook/lib/python3.11/site-packages/xarray/conventions.py:224: SerializationWarning: variable gcm_model has data in the form of a dask array with dtype=object, which means it is being loaded int

In [22]:
file_size = os.path.getsize(outputs[0])
print("File Size is :", file_size/(1024*1024), "MB")

File Size is : 0.186981201171875 MB


In [23]:
ds = xr.open_dataset(outputs[0])
ds

<xarray.Dataset>
Dimensions:          (lat: 10, bnds: 2, lon: 10, time: 24, member: 30)
Coordinates:
  * lat              (lat) float64 50.5 51.5 52.5 53.5 ... 56.5 57.5 58.5 59.5
  * lon              (lon) float64 0.5 1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5
  * time             (time) datetime64[ns] 2013-01-01 2013-02-01 ... 2014-12-01
    member_id        (member) object ...
    gcm_institution  (member) object ...
    gcm_model        (member) object ...
    gcm_variant      (member) object ...
    height2m         float64 ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (time, bnds) datetime64[ns] ...
    t                (member, time, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/27)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-05 15:51:48.916442+01:00
    tracking_id:                4f887b4c-eaff-448c-9e50-4cacd5d16f38
    project_id:                 c3s-atlas-v2